In [1]:
dataset_today =  'Marketing Data.xlsx'

# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
dataset = pd.read_excel(dataset_today)
df = dataset.copy()

# Cleaning Part

### Extract the year and the month columns


In [4]:
df['day'] = pd.to_datetime(df['Date']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['Date']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['Date']).dt.year
y = list(df['year'].dropna().unique())

In [5]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : 'الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : 'بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

### Change the directory in order to put all the excel files reports in the day folder

In [6]:
root_path = 'Desktop/'
os.mkdir(os.path.join(root_path, 'تحليل فروع بالكلمة') )
os.chdir(os.path.join(root_path, 'تحليل فروع بالكلمة' ))

### Prepare the required Columns

In [7]:
df['discount amount'] = df['Quantity'] * df['Discount per Unit (abs)']
df['discount percentage'] = ((df['Discount (%)'] / 100) * df['Unit Price']) *df['Quantity']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

### Prepare different pages 

#### Page of Deposits

In [8]:
df_deposit = df[df['Product Category'].str.contains( 'منتجات مصروفات الشركة')]
df.loc[df_deposit.index,"Quantity"] = 1

#### Page of services and installment

In [9]:
df_services = df[df['Product Category'].str.contains('خدمات-44 ')]

In [10]:
df_installment = df[df['Product Category'].str.contains( 'خدمات التقسيط')]
df_services =pd.concat([df_services, df_installment])

#### Get the additional discounts rows, put it in dataframe and save it as a file

In [11]:
df_all = df[df['Product Category'].str.contains( 'All')]
df_addition_discount = df_all[df_all['Product Category'].str.contains('[11223344] RS خصم اضافي للعميل')]
df_all.drop(df_addition_discount.index, inplace = True)
df_services = pd.concat([df_services, df_all])
df_addition_discount.to_excel('addition_discount report.xlsx',  index = False)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_4988\2280359067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(df_addition_discount.index, inplace = True)


#### Page of zero qunatity

In [12]:
df_0 = df[df['Quantity'] == 0 ]
df.drop(df_0.index, inplace = True)

# Analytical Part

## Product Reports

### Drop additional discounts, services, and deposits from dataframe in order to get accurate results

In [13]:
df2 = df.copy()
df2.drop(df_addition_discount.index, inplace = True)
df2.drop(df_services.index, errors= 'ignore', inplace = True)
df2.drop(df_deposit.index, errors= 'ignore', inplace = True)
df2.dropna(inplace = True)

### Barcode DataFrame

In [14]:
barcode = df[['Product','Barcode']].drop_duplicates()
barcode.rename(columns = {'Product' : 'المنتج', 'Barcode': 'باركود'},inplace = True)

### Category Per day 

In [15]:
df['Month_day'] = df['day'].astype(str) +' '+ df['month']
word = input('write the word you want to filter based on')
word_df = df[df['Product Category'].str.contains(word)].copy()
Month_day = list(word_df['Month_day'].unique())

write the word you want to filter based onشتو


In [16]:
Data_all = pd.DataFrame()
for d in Month_day:
    day =  word_df[word_df['Month_day'] == d]
    branches = list(day['فريق المبيعات'].unique())
    for b in branches:
        branch =  day[day['فريق المبيعات'] == b]
        df_sales = pd.pivot_table(branch, index=['فريق المبيعات','Product'], values=('Subtotal'), aggfunc = 'sum')
        df_sales.reset_index(inplace = True)
        df_sales.rename(columns ={'Subtotal':'إجمالي المبيعات',
                                  "Product": 'المنتج'}, inplace = True) 

        df_sold_units = pd.pivot_table(branch,index=['فريق المبيعات' ,'Product'], values=('Quantity'), aggfunc = 'sum')
        df_sold_units.reset_index(inplace = True)
        df_sold_units.rename(columns ={'Quantity':'عدد الوحدات',
                                       "Product": 'المنتج'}, inplace = True)
        
        
        all_data = df_sales.merge(df_sold_units, on= ['فريق المبيعات' ,'المنتج'], how = 'inner')
        all_data = all_data.merge(barcode, on='المنتج' , how = 'inner')
        
        all_data.loc['الاجمالي', 'عدد الوحدات'] =  df_sold_units['عدد الوحدات'].sum()
        all_data.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()
        all_data.loc['', :] = ''
        all_data.loc[' ', :] = ' '
        all_data.loc[' ',  all_data.columns] = all_data.columns 
        
        Data_all = pd.concat([Data_all, all_data])
        Data_all = Data_all[['فريق المبيعات',
                    'باركود',
                    'المنتج',
                    'عدد الوحدات',
                    'إجمالي المبيعات']]        
    if os.path.exists(('منتجات شتوية' + '.xlsx')) == False:
        Data_all.set_index('فريق المبيعات')
        Data_all.to_excel(('منتجات شتوية باليوم' +'.xlsx'), sheet_name= d) 
    else:    
        with pd.ExcelWriter(('منتجات شتوية باليوم' + '.xlsx'), engine='openpyxl', mode='a') as writer:
            Data_all.to_excel(writer, sheet_name= d)  
    Data_all = pd.DataFrame()


### Category Per branch 


In [17]:
df['Month_day'] = df['day'].astype(str) +' '+ df['month']
branches = list(df['فريق المبيعات'].unique())
word = input('write the word you want to filter based on')
word_df = df[df['Product Category'].str.contains(word)].copy()
Month_day = list(word_df['Month_day'].unique())

write the word you want to filter based onشتو


In [18]:
for b in branches:
    branch =  word_df[word_df['فريق المبيعات'] == b]
    df_sales = pd.pivot_table(branch, index=['Product'], values=('Subtotal'), aggfunc = 'sum')
    df_sales.reset_index(inplace = True)
    df_sales.rename(columns ={'Subtotal':'قمية القطع المباعة',
                              "Product": 'المنتج'}, inplace = True) 

    df_sold_units = pd.pivot_table(branch,index=['Product'], values=('Quantity'), aggfunc = 'sum')
    df_sold_units.reset_index(inplace = True)
    df_sold_units.rename(columns ={'Quantity':'عدد القطع المباعة',
                                   "Product": 'المنتج'}, inplace = True)
        
        
    all_data = df_sales.merge(df_sold_units, on= ['المنتج'], how = 'inner')
    all_data = all_data.merge(barcode, on='المنتج' , how = 'inner')
    try:
        all_data = all_data[all_data['قمية القطع المباعة'] > 0]
        all_data = all_data.sort_values(by= ['قمية القطع المباعة', 'عدد القطع المباعة'], ascending = False)
        all_data['نسبة من إجمالي المبيعات'] = all_data['قمية القطع المباعة'] / all_data['قمية القطع المباعة'].sum()
        all_data = all_data[['المنتج',
                             'باركود',
                             'عدد القطع المباعة',
                             'قمية القطع المباعة',
                             'نسبة من إجمالي المبيعات'
                            ]]
        
        all_data.set_index('المنتج', inplace = True)
    except:
        continue
    all_data.loc['الاجمالي', 'عدد القطع المباعة'] =  df_sold_units['عدد القطع المباعة'].sum()
    all_data.loc['الاجمالي', 'قمية القطع المباعة'] = df_sales['قمية القطع المباعة'].sum()
    all_data.loc['الاجمالي', 'نسبة من إجمالي المبيعات'] = all_data['نسبة من إجمالي المبيعات'].sum()
    all_data.loc['الاجمالي', 'باركود'] = df_sold_units['عدد القطع المباعة'].count()
    
    
    if os.path.exists(('منتجات شتوية بالفرع' + '.xlsx')) == False:
        all_data.to_excel(('منتجات شتوية بالفرع' +'.xlsx'), sheet_name= b) 
    else:    
        with pd.ExcelWriter(('منتجات شتوية بالفرع' + '.xlsx'), engine='openpyxl', mode='a') as writer:
            all_data.to_excel(writer, sheet_name= b)  